In [2]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
from tensorflow.contrib import learn

In [3]:
data_dir = 'data'
ckpt_dir = 'checkpoint'
word_embd_dir = 'checkpoint/word_embd_100'
model_dir = 'checkpoint/model'

In [4]:
f = open('train_data.pkl', 'rb')
data, relation, e1, e2, e1_pos, e2_pos = pickle.load(f)

In [5]:
max_document_length = max([len(x.split(" ")) for x in data])

In [6]:
max_document_length

97

In [7]:
word_embd_dim = 100
pos_embd_dim = 25
dep_embd_dim = 25
vocab_size = 400000
pos_size = 10
relation_classes = 19
state_size = 100
max_sequence_length = 97
batch_size = 10

In [7]:
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(os.path.join(data_dir, "word_vocab100"))

In [23]:
with tf.name_scope("input"):
    sequence_length = tf.placeholder(tf.int32, shape=[1])
    word_ids = tf.placeholder(tf.int32, shape=[max_sequence_length])
#     pos_tag = tf.placeholder(tf.int32, [None, 1], name="pos_tag")
#     relation = tf.placeholder(tf.int32, [None, relation_classes], name="relation")

In [24]:
with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embedded_word = tf.nn.embedding_lookup(W, word_ids)
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

In [25]:
# with tf.name_scope("pos_embedding"):
#     W = tf.Variable(tf.random_uniform([pos_size, pos_embd_dim], name="W"))
#     embedded_pos = tf.nn.embedding_lookup(W, pos_tag)
#     pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

In [26]:
hidden_state = tf.zeros([1, state_size], name='hidden_state')
cell_state = tf.zeros([1, state_size], name='cell_state')

In [27]:
hidden_states = []
cell_states = []

In [28]:
embedded_word

<tf.Tensor 'word_embedding_2/embedding_lookup:0' shape=(97, 100) dtype=float32>

In [13]:
import lstm
lstm.lstm_weights("word", word_embd_dim, state_size)

In [43]:
embd_word_series = tf.split(embedded_word, sequence_length, 0)

In [44]:
for i in range(max_sequence_length):
    if(sequence_length==i):
        l = i

In [45]:
embd_word_series

[<tf.Tensor 'split_1:0' shape=(?, ?) dtype=float32>]

In [46]:
embedded_word

<tf.Tensor 'word_embedding_2/embedding_lookup:0' shape=(97, 100) dtype=float32>

In [24]:
embd_word_series

[<tf.Tensor 'split:0' shape=(?, ?, ?) dtype=float32>]

In [30]:
for i in range(l):
    hidden_state, cell_state = lstm.lstm_cell("word", embd_word[i,:,:], hidden_state, cell_state)
    if i==0:
        hidden_states = hidden_state
    else:
        hidden_states = tf.concat([hidden_states, hidden_state], 0)
    l = l+1

TypeError: 'Tensor' object cannot be interpreted as an integer

In [41]:
# for i in range(2):
#     if i==0:
#         hidden_states = hidden_state
#     else:
#         hidden_states = tf.concat([hidden_states, hidden_state], 0)

In [42]:
hidden_states

<tf.Tensor 'concat_4:0' shape=(2, 100) dtype=float32>

In [27]:
states = tf.concat([hidden_state, cell_state], 1)

In [28]:
cell_state

<tf.Tensor 'word_1/add_8:0' shape=(?, 100) dtype=float32>

In [29]:
states

<tf.Tensor 'concat_1:0' shape=(?, 200) dtype=float32>

In [47]:
sess  =tf.Session()
sess.run(tf.global_variables_initializer())

In [48]:
# sess.run(embedding_init, feed_dict={embedding_placeholder: word_embd})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')


In [49]:
latest_embd = tf.train.latest_checkpoint(word_embd_dir)
word_embedding_saver.restore(sess, latest_embd)

INFO:tensorflow:Restoring parameters from checkpoint/word_embd_100/word_embd


In [1]:
import nltk

In [31]:
f = open('data/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

In [33]:
file = open('data/word_embedding', 'rb')
word_embedding = pickle.load(file)
file.close()

In [37]:
np.shape(word_embedding)

(400000, 100)

In [38]:
word_embedding.append(np.random.normal(size=word_embd_dim,loc=0,scale=0.05))


In [42]:
file = open('data/word_embedding', 'wb')
pickle.dump(word_embedding, file)

In [9]:
word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

In [26]:
word_vocab_size = 400001

In [28]:
word2id["UNKNOWN_TOKEN"] = word_vocab_size
id2word[word_vocab_size-1] = "UNKNOWN_TOKEN"

In [12]:
data

['The system as described above has its greatest application in an arrayed configuration of antenna elements .',
 'The child was carefully wrapped and bound into the cradle by means of a cord .',
 'The author of a keygen uses a disassembler to look at the raw assembly code .',
 'A misty ridge uprises from the surge .',
 'The student association is the voice of the undergraduate student population of the State University of New York at Buffalo .',
 "This is the sprawling complex that is Peru 's largest producer of silver .",
 'The current view is that the chronic inflammation in the distal part of the stomach caused by Helicobacter pylori infection results in an increased acid production from the non-infected upper corpus region of the stomach .',
 'People have been moving back into downtown .',
 'The lawsonite was contained in a platinum crucible and the counter-weight was a plastic crucible with metal pieces .',
 'The solute was placed inside a beaker and 5 mL of the solvent was pipet

In [43]:
tokenized_sentences = [nltk.word_tokenize(line.lower()) for line in data]

In [44]:
unknown_token = "UNKNOWN_TOKEN"

In [52]:
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word2id else unknown_token for w in sent]

In [55]:
tokenized_sentences

[['the',
  'system',
  'as',
  'described',
  'above',
  'has',
  'its',
  'greatest',
  'application',
  'in',
  'an',
  'arrayed',
  'configuration',
  'of',
  'antenna',
  'elements',
  '.'],
 ['the',
  'child',
  'was',
  'carefully',
  'wrapped',
  'and',
  'bound',
  'into',
  'the',
  'cradle',
  'by',
  'means',
  'of',
  'a',
  'cord',
  '.'],
 ['the',
  'author',
  'of',
  'a',
  'UNKNOWN_TOKEN',
  'uses',
  'a',
  'disassembler',
  'to',
  'look',
  'at',
  'the',
  'raw',
  'assembly',
  'code',
  '.'],
 ['a', 'misty', 'ridge', 'UNKNOWN_TOKEN', 'from', 'the', 'surge', '.'],
 ['the',
  'student',
  'association',
  'is',
  'the',
  'voice',
  'of',
  'the',
  'undergraduate',
  'student',
  'population',
  'of',
  'the',
  'state',
  'university',
  'of',
  'new',
  'york',
  'at',
  'buffalo',
  '.'],
 ['this',
  'is',
  'the',
  'sprawling',
  'complex',
  'that',
  'is',
  'peru',
  "'s",
  'largest',
  'producer',
  'of',
  'silver',
  '.'],
 ['the',
  'current',
  'view

In [57]:
word_train = np.asarray([[word2id[w] for w in sentence] for sentence in tokenized_sentences])
